# Datos en Docker

## Bind mounts

Los contenedores son entidades autocontenidas que no pueden acceder por defecto al SO de la maquina que los esta ejecutando, de hecho ellos no saben que estan dentro de una maquina desde el punto de vista del contenedor.

Sin embargo muchas veces, es necesario guardar los datos del contenedor, para usarlos en otros procesos. 

Crearemos un nuevo folder llamado *myDockerData*

Para esta practica crearemos un container basado MongoDB:

    sudo docker run --name myMongoPlatzi --detach mongo:5.0

Hubiera podido colocar solo *mongo* al final pero como ya tenia una imagen guardada, para hacerlo mas rapido, baso mi nuevo conteneder en la existente.

Verifico este corriendo:

    sudo docker ps

Para generar algunso datos dentro del contenedor, primero executamos una terminal:

    sudo docker exec -it  myMongoPlatzi bash

El prompt cambia, indicando podemos ejecutar comandos dentro del contenedor:

    root@4e10d97111ad:/#

Este contenedor viene con un binario que es el cliente de la base de datos, 

    mongo

Tambien puedo usar *mongosh* como es constumbre en el curso de MongoDB

    mongosh


Y ahora, algunso comandos de Mongo para crear una base de datos, la coleccion y almenos un documento.

    use platzi
    db.user.insert({ name: "rodrigo", age: 30 })

Verificando este creado:

    db.user.insert({ name: "rodrigo", age: 30 })

Sin embargo, cuando paremos el contenedor, y cuando lo borremos, perderemos la informacion. 😞

### Creando el bind mount

Una forma de no perder la informacion, es tener un directorio en mi maquina que tenga una version espejada de lo que ocurre en el contenedor. Para hacer esto haremos un *bind mount*

Borremos el contenedor y creemos uno nuevo con el mismo nombre:

    sudo docker rm myMongoPlatzi


con *-v* o *--volume* le indicamos que queremos hacer un *bind mount*. Mongo guarda los datos en **/data/db**. Entonces iria asi 

**--volume (ruta local):(ruta a copiar)**

    docker run -d --name myMongoPlatzi --volume /home/carlos/Documents/curso_docker/myDockerData:/data/db mongo:5.0

De igual manera, volvemos a crear los datos nuevamente

    mongosh

    use platzi

    db.users.insertMany([{ name: "gustavo", age: 31}, { name: "rodrigo", age:19 }])

    db.users.find()

Salgamos, y borremos el contenedor totalmente

    sudo docker rm -f myMongoPlatzi

Observemos que la carpeta *myDockerData* tiene nueva informacion:

![](https://i.imgur.com/cTrq831.png)


### Recuperando los datos

Volvamos a crear un nuevo contenedor con el nombre *osito*, teniendo en cuenta de montar el directorio, que ya tiene nueva informacion:

    docker run -d --name osito --volume /home/carlos/Documents/curso_docker/myDockerData:/data/db mongo:5.0

Y efectivamente al hacer todo el procedimiento, y mirar con el querry:

    use platzi
    db.users.find()

Y estaran los datos tal y como los habia guardado 😊😊


Esta es una manera muy practica de compartir los datos de mi disco hacia el contenedor, sin embargo esto tiene un riesgo, al darle acceso irrestrico de una parte del disco al contenedor, pero no es la practica mas segura.  